# Automated Visual Weak Supervision for Object Recognition in Videos

In [56]:
%load_ext autoreload
%autoreload 2

import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

from process_tubes import *
from reef_label_tubes import reef_label
import ClassifierLoader
from tube_classifier import *

Using device: cuda
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Split the Dataset

Here, we split our dataset into an unlabeled train set (30 objects), a labeled validation set (300), and a labeled test set (70). Each object maps to an action tube consisting of 30 frames.

In [57]:
labeled_vehicles, unlabeled_vehicles, test_vehicles = get_objects((30, 300, 70))

print(labeled_vehicles.shape[0], 'labeled train examples')
print(unlabeled_vehicles.shape[0], 'unlabeled train examples')
print(test_vehicles.shape[0], 'labeled test examples')

30 labeled train examples
300 unlabeled train examples
70 labeled test examples


## 2. Create Action Tubes for Weak Supervision

At this step, we read in our frames, encode them with our optimized autoencoder, and concatenate them to make action tubes.

In [58]:
val_primitive_matrix, val_tubes = tube_loader(labeled_vehicles, label = True)
val_ground = [label for tube in val_tubes for label in tube.tube_true_labels]

Using device: cuda
Producing action tubes...
Creating action tube for object index 4061
Creating action tube for object index 3051
Creating action tube for object index 3807
Creating action tube for object index 3255
Creating action tube for object index 1652
Creating action tube for object index 3696
Creating action tube for object index 4036
Creating action tube for object index 2104
Creating action tube for object index 1914
Creating action tube for object index 1596
Creating action tube for object index 2312
Creating action tube for object index 547
Creating action tube for object index 3811
Creating action tube for object index 640
Creating action tube for object index 2226
Creating action tube for object index 4001
Creating action tube for object index 3164
Creating action tube for object index 2946
Creating action tube for object index 507
Creating action tube for object index 3782
Creating action tube for object index 3347
Creating action tube for object index 3968
Creating act

In [ ]:
train_primitive_matrix, train_tubes = tube_loader(unlabeled_vehicles)

Producing action tubes...
Creating action tube for object index 2284
Creating action tube for object index 2500
Creating action tube for object index 3048
Creating action tube for object index 705
Creating action tube for object index 4035
Creating action tube for object index 3994
Creating action tube for object index 3447
Creating action tube for object index 1306
Creating action tube for object index 3113
Creating action tube for object index 2427
Creating action tube for object index 3186
Creating action tube for object index 2930
Creating action tube for object index 3960
Creating action tube for object index 3829
Creating action tube for object index 645
Creating action tube for object index 3085
Creating action tube for object index 2018
Creating action tube for object index 1231
Creating action tube for object index 3343
Creating action tube for object index 3056
Creating action tube for object index 3478
Creating action tube for object index 188
Creating action tube for object

Creating action tube for object index 259
Creating action tube for object index 1639
Creating action tube for object index 2619
Creating action tube for object index 3124
Creating action tube for object index 3815
Creating action tube for object index 3622
Creating action tube for object index 3444
Creating action tube for object index 3869
Creating action tube for object index 879
Creating action tube for object index 3603
Creating action tube for object index 3252
Creating action tube for object index 447
Creating action tube for object index 3604
Creating action tube for object index 4145
Creating action tube for object index 3381
Creating action tube for object index 3822
Creating action tube for object index 3377
Creating action tube for object index 1658
Creating action tube for object index 3345
Creating action tube for object index 125
Creating action tube for object index 295
Creating action tube for object index 4175
Creating action tube for object index 170
Creating action t

## 3. Apply Visual Weak Supervision

To perform visual weak supervision, we pass our frame encodings from the unlabeled train set and our labeled validation set into Reef. Note that we have already tested various methods for weak supervision, and so the `train_ground` parameter is not used here. Please refer to the `generate*.ipynb` notebooks to see our weak supervision experiments.

In [ ]:
unlabeled_frame_nums = [frame_num for tube in train_tubes for frame_num in tube.sampled_frames]
weak_train = reef_label(train_primitive_matrix, val_primitive_matrix, val_ground, None, unlabeled_frame_nums)

Running iteration:  5
Running iteration:  10
Running iteration:  15
Running iteration:  20


## 4. Weakly Label Training Tubes

In the previous step, we generate probabilistic weak labels for each frame. Here, we aggregate all the frame-level labels for all frames in the tube and assign the tube the majority label. This step is necessary for future tube-level classification.

In [ ]:
for tube in train_tubes:
    tube.assign_label(weak_train)
    print(tube.pred_vehicle, tube.true_vehicle)
    
for tube in val_tubes:
    tube.true_vehicle = 0 if tube.true_vehicle == -1 else 1
    print(tube.true_vehicle)

1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None
1 None

## 5. Run Classifier

Here, we demonstrate hyperparameter tuning with our final 3D-CNN classifier built on our weakly-labeled tubes. This classifier assigns a given tube either a 1 (for car) or a 0 (for truck). We have conducted several experiments that involve varying the volume of weakly-labeled data passed into the classifier and monitor relative performance.

### Experiments:
1. Learning rate modulation over 5 epochs
2. Weak volume: 0
2. Weak volume: 100
3. Weak volume: 200
4. Weak volume: 300

In [ ]:
# val_data = ClassifierLoader.ClassifierLoader(val_primitive_matrix, val_tubes, 'val')
# val_data[0]

# train_data = ClassifierLoader.ClassifierLoader(train_primitive_matrix, train_tubes, 'train')
# train_data[0]

tune(train_primitive_matrix, train_tubes, val_primitive_matrix, val_tubes)

Epoch 0
Epoch Loss = 0.52377
Classified 20 / 30 correctly (66.67)
Epoch 1
Epoch Loss = 0.64830
Classified 20 / 30 correctly (66.67)
Epoch 2
Epoch Loss = 0.64450
Classified 20 / 30 correctly (66.67)
Epoch 3
Epoch Loss = 0.64021
Classified 20 / 30 correctly (66.67)
Epoch 4
Epoch Loss = 0.63544
Classified 20 / 30 correctly (66.67)
Final Loss = 0.63544
Learning Rate = 0.00001, loss = 0.6354
Epoch 0
Epoch Loss = 0.50699
Classified 20 / 30 correctly (66.67)
Epoch 1
Epoch Loss = 0.62759
Classified 20 / 30 correctly (66.67)
Epoch 2
Epoch Loss = 0.56297
Classified 20 / 30 correctly (66.67)
Epoch 3
Epoch Loss = 0.50986
Classified 20 / 30 correctly (66.67)
Epoch 4
Epoch Loss = 0.46494
Classified 20 / 30 correctly (66.67)
Final Loss = 0.46494
Learning Rate = 0.00010, loss = 0.4649
Epoch 0
Epoch Loss = 0.31345
Classified 20 / 30 correctly (66.67)
Epoch 1
Epoch Loss = 0.45982
Classified 20 / 30 correctly (66.67)
Epoch 2
Epoch Loss = 0.33986
Classified 20 / 30 correctly (66.67)
Epoch 3
Epoch Loss = 0